
Data Cleaning Questions:


In [ ]:
import pandas as pd

In [ ]:
data = r"/content/Electric_Vehicle_Population_Data.csv"

In [ ]:
df = pd.read_csv(data)

In [ ]:
missing = df.isnull().sum()

In [ ]:
print(missing)

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df['Electric Range'] = df['Electric Range'].replace(0, pd.NA)

In [ ]:
df.head()

In [ ]:
df['Electric Range'].fillna(df['Electric Range'].median(), inplace=True)

In [ ]:
df.head()

In [ ]:
A=df.duplicated().sum()

In [ ]:
print("Number of rows :",A)

In [ ]:
df = df.drop_duplicates

In [ ]:
import pandas as pd

In [ ]:
import hashlib

In [ ]:
data=r"/content/Electric_Vehicle_Population_Data.csv"

In [ ]:
df = pd.read_csv(data)

In [ ]:
type(df)

In [ ]:
print(df.columns)

In [ ]:
def anonymize_vin(vin):
    if pd.isna(vin):
        return None
    return hashlib.sha256(str(vin).encode()).hexdigest()

df['ANONYMIZE'] = df['VIN (1-10)'].apply(anonymize_vin)


In [ ]:
df['Vehicle Location'] = (
    df['Vehicle Location']
    .astype(str)
    .str.replace('POINT', '', regex=False)
    .str.replace('[()]', '', regex=True)
    .str.strip()
)


In [ ]:
a = df['Vehicle Location'].str.split(r'\s+', expand=True)


In [ ]:
df['Longitude'] = pd.to_numeric(a[0], errors='coerce')
df['Latitude'] = pd.to_numeric(a[1], errors='coerce')


In [ ]:
df = df.dropna(subset=['Latitude', 'Longitude'])


. Data Exploration


In [ ]:
top_5_makes = df['Make'].value_counts().head(5)
top_5_models = df['Model'].value_counts().head(5)

top_5_makes, top_5_models


In [ ]:
county_distribution = df['County'].value_counts()
top_county = county_distribution.idxmax()

county_distribution.head(), top_county


In [ ]:
ev_by_year = df['Model Year'].value_counts().sort_index()
ev_by_year


In [ ]:
average_electric_range = df['Electric Range'].mean()
average_electric_range


In [ ]:
cafv_percentage = (
    df['Clean Alternative Fuel Vehicle (CAFV) Eligibility']
    .value_counts(normalize=True) * 100
)

cafv_percentage


In [ ]:
avg_range_by_make = (
    df.groupby('Make')['Electric Range']
    .mean()
    .sort_values(ascending=False)
)

avg_range_by_model = (
    df.groupby('Model')['Electric Range']
    .mean()
    .sort_values(ascending=False)
)

avg_range_by_make, avg_range_by_model


In [ ]:
df['County'].value_counts().head(10)


Data Visualization Questions

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium


In [ ]:
# Top 5 Makes
top_makes = df['Make'].value_counts().head(5)
plt.figure(figsize=(8,5))
sns.barplot(x=top_makes.index, y=top_makes.values, palette="viridis")
plt.title("Top 5 EV Makes by Count")
plt.ylabel("Number of EVs")
plt.xlabel("Make")
plt.show()

# Top 5 Models
top_models = df['Model'].value_counts().head(5)
plt.figure(figsize=(10,5))
sns.barplot(x=top_models.index, y=top_models.values, palette="magma")
plt.title("Top 5 EV Models by Count")
plt.ylabel("Number of EVs")
plt.xlabel("Model")
plt.xticks(rotation=45)
plt.show()


In [ ]:
# County distribution
county_counts = df['County'].value_counts().reset_index()
county_counts.columns = ['County', 'EV_Count']

# Plot heatmap using Seaborn (simple version)
plt.figure(figsize=(12,6))
sns.heatmap(county_counts.set_index('County').T, annot=True, cmap="YlGnBu")
plt.title("EV Distribution by County")
plt.show()


In [ ]:
ev_by_year = df['Model Year'].value_counts().sort_index()
plt.figure(figsize=(10,5))
sns.lineplot(x=ev_by_year.index, y=ev_by_year.values, marker="o")
plt.title("EV Adoption Trend by Model Year")
plt.xlabel("Model Year")
plt.ylabel("Number of EVs")
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=df, x='Electric Range',  hue='Make', palette='tab10', alpha=0.7)

plt.xlabel("Electric Range (miles)")

plt.legend(title='Make')
plt.show()


In [ ]:
cafv_counts = df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].value_counts()
plt.figure(figsize=(6,6))
plt.pie(cafv_counts, labels=cafv_counts.index, autopct='%1.1f%%', startangle=140, colors=['#66b3ff','#ff9999'])
plt.title("CAFV Eligibility Proportion")
plt.show()


In [ ]:
# Ensure Latitude & Longitude columns exist
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=6)

for idx, row in df.iterrows():
    if pd.notnull(row['Latitude']) and pd.notnull(row['Longitude']):
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=3,
            color='blue',
            fill=True,
            fill_opacity=0.6
        ).add_to(m)

# Display map
m
